# Exercices : Cryptographie à Clé Publique

**Objectifs** :
- Attaquer RSA textbook
- Exploiter les faiblesses de DH
- Comprendre les vulnérabilités d'ECDSA
- Implémenter un chiffrement hybride

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa, dh, ec, padding
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.backends import default_backend
import secrets
import hashlib

## Exercice 1 : Attaque sur RSA avec Petits Premiers

**Contexte** : Un système RSA mal implémenté utilise de petits nombres premiers.

**Tâche** : Factorisez $n$ et récupérez la clé privée.

In [ ]:
def small_primes_attack():
    """
    Attaque sur RSA avec petits premiers (éducatif).
    """
    print("=" * 70)
    print("ATTAQUE : RSA avec Petits Premiers")
    print("=" * 70)
    
    # RSA vulnérable avec petits premiers
    p = 65537  # Premier trop petit !
    q = 65539  # Autre premier trop petit !
    
    n = p * q
    phi_n = (p - 1) * (q - 1)
    e = 65537
    d = pow(e, -1, phi_n)
    
    print(f"\n🔐 RSA vulnérable :")
    print(f"   p = {p}")
    print(f"   q = {q}")
    print(f"   n = {n} ({n.bit_length()} bits)")
    print(f"   e = {e}")
    
    # Message chiffré
    message = 12345
    ciphertext = pow(message, e, n)
    
    print(f"\n📝 Message : {message}")
    print(f"   Chiffré : {ciphertext}")
    
    # Attaque : Factorisation par force brute
    print(f"\n💥 Attaque : Factorisation de n = {n}")
    print(f"   Recherche de facteurs...")
    
    # Méthode simple : essayer tous les diviseurs
    def factorize(n):
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                return i, n // i
        return None, None
    
    p_found, q_found = factorize(n)
    
    if p_found and q_found:
        print(f"\n✅ Facteurs trouvés :")
        print(f"   p = {p_found}")
        print(f"   q = {q_found}")
        
        # Récupérer clé privée
        phi_n_recovered = (p_found - 1) * (q_found - 1)
        d_recovered = pow(e, -1, phi_n_recovered)
        
        print(f"\n🔓 Clé privée récupérée :")
        print(f"   d = {d_recovered}")
        print(f"   Correspondance : {d_recovered == d} ✅")
        
        # Déchiffrement
        decrypted = pow(ciphertext, d_recovered, n)
        print(f"\n📖 Message déchiffré : {decrypted}")
        print(f"   Correct : {decrypted == message} ✅")
    
    print(f"\n⚠️  LEÇON : TOUJOURS utiliser de grands premiers")
    print(f"   Minimum : 1024 bits par premier (RSA-2048 total)")
    print(f"   Recommandé : 1536 bits par premier (RSA-3072 total)")

small_primes_attack()

## Exercice 2 : Common Modulus Attack (RSA)

**Contexte** : Deux utilisateurs partagent le même modulus $n$ mais ont des exposants $e_1, e_2$ différents.

**Tâche** : Si le même message est chiffré pour les deux, récupérez-le sans les clés privées !

In [ ]:
def common_modulus_attack():
    """
    Attaque Common Modulus sur RSA.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE : Common Modulus (RSA)")
    print("=" * 70)
    
    # Setup vulnérable : même n, différents e
    p = 61
    q = 53
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    # Deux utilisateurs
    e1 = 17
    e2 = 23
    d1 = pow(e1, -1, phi_n)
    d2 = pow(e2, -1, phi_n)
    
    print(f"\n🔐 Setup vulnérable :")
    print(f"   n commun : {n}")
    print(f"   Utilisateur 1 : e1 = {e1}, d1 = {d1}")
    print(f"   Utilisateur 2 : e2 = {e2}, d2 = {d2}")
    
    # Message envoyé aux deux
    message = 42
    c1 = pow(message, e1, n)
    c2 = pow(message, e2, n)
    
    print(f"\n📝 Même message chiffré pour les deux :")
    print(f"   m = {message}")
    print(f"   c1 = m^e1 mod n = {c1}")
    print(f"   c2 = m^e2 mod n = {c2}")
    
    # Attaque : Utiliser Bezout (e1·a + e2·b = gcd(e1, e2) = 1)
    print(f"\n💥 Attaque (sans clés privées) :")
    
    # Algorithme d'Euclide étendu
    def extended_gcd(a, b):
        if b == 0:
            return a, 1, 0
        gcd, x1, y1 = extended_gcd(b, a % b)
        x = y1
        y = x1 - (a // b) * y1
        return gcd, x, y
    
    gcd, a, b = extended_gcd(e1, e2)
    
    print(f"   gcd({e1}, {e2}) = {gcd}")
    print(f"   Identité de Bezout : {e1}·{a} + {e2}·{b} = {gcd}")
    
    if gcd == 1:
        # m = (c1^a · c2^b) mod n
        # Car c1^a · c2^b = m^(e1·a) · m^(e2·b) = m^(e1·a + e2·b) = m^1 = m
        
        if a < 0:
            c1_inv = pow(c1, -1, n)
            m_recovered = (pow(c1_inv, -a, n) * pow(c2, b, n)) % n
        elif b < 0:
            c2_inv = pow(c2, -1, n)
            m_recovered = (pow(c1, a, n) * pow(c2_inv, -b, n)) % n
        else:
            m_recovered = (pow(c1, a, n) * pow(c2, b, n)) % n
        
        print(f"\n✅ Message récupéré : {m_recovered}")
        print(f"   Message original : {message}")
        print(f"   Correspondance : {m_recovered == message} ✅")
    
    print(f"\n⚠️  LEÇON : NE JAMAIS partager n entre utilisateurs !")
    print(f"   Chaque utilisateur doit avoir son propre (p, q, n)")

common_modulus_attack()

## Exercice 3 : MITM sur Diffie-Hellman Non Authentifié

**Contexte** : DH sans authentification est vulnérable à Man-in-the-Middle.

**Tâche** : Implémentez l'attaque MITM complète.

In [ ]:
def mitm_dh_full_attack():
    """
    Attaque MITM complète sur DH.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE MITM : Diffie-Hellman Non Authentifié")
    print("=" * 70)
    
    # Paramètres DH
    parameters = dh.generate_parameters(generator=2, key_size=512, backend=default_backend())
    
    # Alice et Bob
    print("\n👩 Alice génère sa paire DH")
    alice_private = parameters.generate_private_key()
    alice_public = alice_private.public_key()
    
    print("👨 Bob génère sa paire DH")
    bob_private = parameters.generate_private_key()
    bob_public = bob_private.public_key()
    
    # Eve (attaquante MITM)
    print("\n💀 Eve se positionne en Man-in-the-Middle")
    eve_private_for_alice = parameters.generate_private_key()
    eve_public_for_alice = eve_private_for_alice.public_key()
    
    eve_private_for_bob = parameters.generate_private_key()
    eve_public_for_bob = eve_private_for_bob.public_key()
    
    # Échanges interceptés
    print("\n📡 Échanges (tous interceptés par Eve) :")
    print("   1. Alice → Eve (pense : Alice → Bob) : A_pub")
    print("   2. Eve → Bob (prétend être Alice) : E1_pub")
    print("   3. Bob → Eve (pense : Bob → Alice) : B_pub")
    print("   4. Eve → Alice (prétend être Bob) : E2_pub")
    
    # Secrets partagés
    alice_shared_with_eve = alice_private.exchange(eve_public_for_alice)
    eve_shared_with_alice = eve_private_for_alice.exchange(alice_public)
    
    bob_shared_with_eve = bob_private.exchange(eve_public_for_bob)
    eve_shared_with_bob = eve_private_for_bob.exchange(bob_public)
    
    print(f"\n🔑 Secrets partagés :")
    print(f"   Alice ↔ Eve : {alice_shared_with_eve.hex()[:32]}...")
    print(f"   Bob ↔ Eve   : {bob_shared_with_eve.hex()[:32]}...")
    print(f"   Alice et Bob n'ont PAS le même secret !")
    
    # Simulation : Alice envoie un message à Bob
    print(f"\n📨 Alice chiffre un message pour 'Bob' :")
    message_alice = b"Secret: Meet at noon"
    
    # Alice chiffre avec son secret (partagé avec Eve)
    key_alice = hashlib.sha256(alice_shared_with_eve).digest()[:16]
    aesgcm_alice = AESGCM(key_alice)
    nonce_alice = secrets.token_bytes(12)
    ciphertext_alice = aesgcm_alice.encrypt(nonce_alice, message_alice, None)
    
    print(f"   Message : {message_alice}")
    print(f"   Chiffré : {ciphertext_alice.hex()[:32]}...")
    
    # Eve intercepte et déchiffre
    print(f"\n💀 Eve intercepte et déchiffre :")
    key_eve_alice = hashlib.sha256(eve_shared_with_alice).digest()[:16]
    aesgcm_eve_alice = AESGCM(key_eve_alice)
    message_intercepted = aesgcm_eve_alice.decrypt(nonce_alice, ciphertext_alice, None)
    
    print(f"   Message déchiffré : {message_intercepted} ✅")
    print(f"   Eve peut LIRE tous les messages !")
    
    # Eve rechiffre pour Bob
    print(f"\n📤 Eve rechiffre pour Bob :")
    message_modified = b"Secret: Meet at MIDNIGHT"  # Eve modifie !
    
    key_eve_bob = hashlib.sha256(eve_shared_with_bob).digest()[:16]
    aesgcm_eve_bob = AESGCM(key_eve_bob)
    nonce_eve = secrets.token_bytes(12)
    ciphertext_for_bob = aesgcm_eve_bob.encrypt(nonce_eve, message_modified, None)
    
    print(f"   Message modifié : {message_modified}")
    
    # Bob déchiffre
    print(f"\n👨 Bob déchiffre :")
    key_bob = hashlib.sha256(bob_shared_with_eve).digest()[:16]
    aesgcm_bob = AESGCM(key_bob)
    message_bob_receives = aesgcm_bob.decrypt(nonce_eve, ciphertext_for_bob, None)
    
    print(f"   Message reçu : {message_bob_receives}")
    print(f"   Bob pense que ça vient d'Alice ❌")
    
    print(f"\n💥 CATASTROPHE :")
    print(f"   - Eve peut LIRE tous les messages")
    print(f"   - Eve peut MODIFIER tous les messages")
    print(f"   - Alice et Bob ne détectent RIEN")
    
    print(f"\n✅ SOLUTION : Authenticated DH")
    print(f"   - Utiliser certificats (TLS)")
    print(f"   - Signer les clés publiques DH")
    print(f"   - Utiliser pre-shared keys")

mitm_dh_full_attack()

## Exercice 4 : Chiffrement Hybride Complet

**Contexte** : Implémentez un système de chiffrement hybride RSA + AES-GCM.

**Tâche** : Créez les fonctions d'encryption et de décryption.

In [ ]:
class HybridCrypto:
    """
    Système de chiffrement hybride RSA-OAEP + AES-GCM.
    """
    
    def __init__(self):
        # Générer paire RSA
        self.private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
            backend=default_backend()
        )
        self.public_key = self.private_key.public_key()
    
    def encrypt(self, plaintext: bytes, recipient_public_key) -> dict:
        """
        Chiffre un message avec chiffrement hybride.
        
        Args:
            plaintext: Message à chiffrer
            recipient_public_key: Clé publique RSA du destinataire
        
        Returns:
            dict avec 'encrypted_key', 'nonce', 'ciphertext', 'tag'
        """
        # 1. Générer clé AES aléatoire
        aes_key = AESGCM.generate_key(bit_length=256)
        
        # 2. Chiffrer message avec AES-GCM
        aesgcm = AESGCM(aes_key)
        nonce = secrets.token_bytes(12)
        ciphertext = aesgcm.encrypt(nonce, plaintext, None)
        
        # 3. Chiffrer clé AES avec RSA-OAEP
        encrypted_key = recipient_public_key.encrypt(
            aes_key,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        
        return {
            'encrypted_key': encrypted_key,
            'nonce': nonce,
            'ciphertext': ciphertext
        }
    
    def decrypt(self, encrypted_data: dict, recipient_private_key) -> bytes:
        """
        Déchiffre un message avec chiffrement hybride.
        
        Args:
            encrypted_data: Dictionnaire retourné par encrypt()
            recipient_private_key: Clé privée RSA du destinataire
        
        Returns:
            Message déchiffré
        """
        # 1. Déchiffrer clé AES avec RSA-OAEP
        aes_key = recipient_private_key.decrypt(
            encrypted_data['encrypted_key'],
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        
        # 2. Déchiffrer message avec AES-GCM
        aesgcm = AESGCM(aes_key)
        plaintext = aesgcm.decrypt(
            encrypted_data['nonce'],
            encrypted_data['ciphertext'],
            None
        )
        
        return plaintext

def hybrid_crypto_demo():
    """
    Démonstration complète du chiffrement hybride.
    """
    print("\n" + "=" * 70)
    print("CHIFFREMENT HYBRIDE : RSA-OAEP + AES-GCM")
    print("=" * 70)
    
    # Alice et Bob ont leurs paires de clés
    alice = HybridCrypto()
    bob = HybridCrypto()
    
    print("\n👩 Alice et 👨 Bob ont généré leurs clés RSA")
    
    # Alice envoie un message à Bob
    message = b"This is a very long confidential message that would be inefficient to encrypt with RSA alone. " * 5
    
    print(f"\n📝 Message d'Alice à Bob :")
    print(f"   Taille : {len(message)} bytes")
    
    # Chiffrement
    print(f"\n🔐 Alice chiffre avec la clé publique de Bob :")
    encrypted = alice.encrypt(message, bob.public_key)
    
    print(f"   Clé AES chiffrée : {len(encrypted['encrypted_key'])} bytes")
    print(f"   Nonce : {encrypted['nonce'].hex()}")
    print(f"   Ciphertext : {len(encrypted['ciphertext'])} bytes")
    
    total_size = len(encrypted['encrypted_key']) + len(encrypted['nonce']) + len(encrypted['ciphertext'])
    print(f"   Total transmis : {total_size} bytes")
    
    # Déchiffrement
    print(f"\n🔓 Bob déchiffre avec sa clé privée :")
    decrypted = bob.decrypt(encrypted, bob.private_key)
    
    print(f"   Taille déchiffrée : {len(decrypted)} bytes")
    print(f"   Correct : {decrypted == message} ✅")
    
    # Test : Mauvaise clé privée
    print(f"\n🔍 Test : Eve essaie de déchiffrer avec sa propre clé :")
    eve = HybridCrypto()
    
    try:
        eve.decrypt(encrypted, eve.private_key)
        print(f"   ❌ ERREUR : Eve a pu déchiffrer !")
    except Exception:
        print(f"   ✅ Déchiffrement échoué (attendu)")
        print(f"   → Seul Bob peut déchiffrer")
    
    # Comparaison tailles
    print(f"\n📊 Comparaison avec RSA pur :")
    rsa_only_blocks = (len(message) + 190) // 190  # ~190 bytes par bloc RSA-OAEP
    rsa_only_size = rsa_only_blocks * 256
    
    print(f"   RSA-OAEP seul : ~{rsa_only_size} bytes ({rsa_only_blocks} blocs)")
    print(f"   Hybride : {total_size} bytes")
    print(f"   Économie : {rsa_only_size - total_size} bytes ({(1 - total_size/rsa_only_size)*100:.1f}%)")

hybrid_crypto_demo()

## Exercice 5 : Récupération de Clé ECDSA (Nonce Leak)

**Contexte** : Simulation de l'attaque PlayStation 3 (nonce constant).

**Tâche** : À partir de deux signatures avec le même nonce, récupérez la clé privée.

In [ ]:
def ecdsa_nonce_reuse_recovery_theory():
    """
    Explication théorique de la récupération de clé ECDSA.
    
    Note : L'implémentation complète nécessiterait d'accéder aux
    valeurs (r, s) brutes, ce qui n'est pas direct avec cryptography.
    """
    print("\n" + "=" * 70)
    print("ATTAQUE : Récupération de Clé ECDSA (Nonce Reuse)")
    print("=" * 70)
    
    print(f"\n📜 Contexte Historique : PlayStation 3 (2010)")
    print(f"   Sony a utilisé un k CONSTANT pour signer tous les jeux")
    print(f"   → Hackers ont récupéré la clé privée")
    print(f"   → Jeux piratés signés comme légitimes")
    
    print(f"\n🔍 Rappel ECDSA :")
    print(f"   Signature : (r, s) où")
    print(f"   r = x-coordinate of kG")
    print(f"   s = k⁻¹(H(m) + dr) mod n")
    
    print(f"\n💥 Attaque (k réutilisé) :")
    print(f"\n   Deux signatures avec MÊME k :")
    print(f"   (r₁, s₁) pour m₁ : s₁ = k⁻¹(H(m₁) + dr₁)")
    print(f"   (r₂, s₂) pour m₂ : s₂ = k⁻¹(H(m₂) + dr₂)")
    print(f"   Avec r₁ = r₂ = r (même k → même r)")
    
    print(f"\n   Étape 1 : Récupérer k")
    print(f"   s₁ - s₂ = k⁻¹(H(m₁) - H(m₂))")
    print(f"   k = (H(m₁) - H(m₂)) / (s₁ - s₂) mod n")
    
    print(f"\n   Étape 2 : Récupérer d (clé privée)")
    print(f"   s₁ = k⁻¹(H(m₁) + dr)")
    print(f"   d = (s₁·k - H(m₁)) / r mod n")
    
    print(f"\n   CLÉ PRIVÉE COMPROMISE ! 💀")
    
    print(f"\n📊 Exemple Numérique Simplifié :")
    
    # Paramètres jouets
    n = 23  # Ordre du groupe (jouet !)
    d = 7   # Clé privée
    k = 5   # Nonce (réutilisé, fatal !)
    r = 13  # r (dépend de k, donc identique)
    
    m1 = 10
    m2 = 15
    h1 = m1  # Hash simplifié
    h2 = m2
    
    # Signatures
    k_inv = pow(k, -1, n)
    s1 = (k_inv * (h1 + d * r)) % n
    s2 = (k_inv * (h2 + d * r)) % n
    
    print(f"\n   Paramètres :")
    print(f"   n = {n}, d = {d} (secret), k = {k} (réutilisé), r = {r}")
    print(f"   m₁ = {m1}, m₂ = {m2}")
    print(f"   s₁ = {s1}, s₂ = {s2}")
    
    # Attaque
    k_recovered = ((h1 - h2) * pow(s1 - s2, -1, n)) % n
    d_recovered = ((s1 * k_recovered - h1) * pow(r, -1, n)) % n
    
    print(f"\n   Récupération :")
    print(f"   k récupéré : {k_recovered} (original : {k}) ✅")
    print(f"   d récupéré : {d_recovered} (original : {d}) ✅")
    
    print(f"\n✅ PROTECTIONS :")
    print(f"   1. k DOIT être aléatoire et unique")
    print(f"   2. Utiliser RFC 6979 (k déterministe mais unique)")
    print(f"   3. Utiliser Ed25519 (k = H(secret || m))")
    print(f"   4. Audits de code cryptographique")

ecdsa_nonce_reuse_recovery_theory()

## Conclusion

**Points clés vérifiés** :
- ❌ RSA textbook : Vulnérable (homomorphisme, déterministe)
- ❌ Petits premiers RSA : Factorisable rapidement
- ❌ Common modulus RSA : Permet récupération de messages
- ❌ DH non authentifié : Vulnérable à MITM
- ❌ ECDSA nonce reuse : Récupération de clé privée
- ✅ Chiffrement hybride : Efficace et sécurisé

**Retenir** :
- Toujours utiliser RSA-OAEP (jamais textbook)
- Toujours authentifier Diffie-Hellman
- Nonce ECDSA doit être aléatoire et unique
- Préférer Ed25519 à ECDSA (moins de risques)
- Chiffrement hybride pour messages longs
- Utiliser bibliothèques auditées (cryptography, OpenSSL)